In [1]:
%matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
# import gmplot
# from geopy.geocoders import Nominatim
# import time
# import gmaps
# import gmaps.datasets

Aijobsindustry_dataset = pd.read_csv("AIJobsIndustry_Clean_v3.csv")
print(f"No of rows in Aijobsindustry_dataset is {Aijobsindustry_dataset.shape[0]}")

Dice_us_jobs_dataset = pd.read_csv("Dice_US_jobs_Clean_v4.csv", encoding="cp437")
print(f"No of rows in Dice_us_jobs_dataset is {Dice_us_jobs_dataset.shape[0]}")

monster_dataset = pd.read_csv("Monster_Clean.csv")
print(f"No of rows in monster_dataset is {monster_dataset.shape[0]}")

indeed_dataset = pd.read_csv("alldata_Clean_v3.csv")
print(f"No of rows in indeed_dataset is {indeed_dataset.shape[0]}")

multi_src_dataset = pd.read_csv("cleanedandfilteredUSJobs.csv")
print(f"No of rows in multiple source data set is {multi_src_dataset.shape[0]}")

##file needs to match format for concat
dice_tech_filtered = pd.read_csv("Dice_US_Tech_FilteredJobData.csv")
print(f"No of rows in dice_tech_filtered is {dice_tech_filtered.shape[0]}")

print(f"No of rows in final merged data set should be {Aijobsindustry_dataset.shape[0]+multi_src_dataset.shape[0]+Dice_us_jobs_dataset.shape[0]+monster_dataset.shape[0]+indeed_dataset.shape[0]+dice_tech_filtered.shape[0]}")

datasets_to_merge = [Aijobsindustry_dataset,Dice_us_jobs_dataset,monster_dataset,indeed_dataset,dice_tech_filtered,multi_src_dataset]

##please confirm the concat makes sense
merged_dataset = pd.concat(datasets_to_merge, sort=False)
print(f"Actual no of rows in merged_dataset is {merged_dataset.shape[0]}")

Using matplotlib backend: Qt5Agg
No of rows in Aijobsindustry_dataset is 631
No of rows in Dice_us_jobs_dataset is 1265
No of rows in monster_dataset is 162
No of rows in indeed_dataset is 6953
No of rows in multiple source data set is 207
No of rows in dice_tech_filtered is 1566
No of rows in final merged data set should be 10784
Actual no of rows in merged_dataset is 10784


In [2]:
merged_dataset.head()

,Position,Company,Description,Loc_state,Loc_city,Skills,Type,Source
0,Data Science Development Program- Junior Data ...,MassMutual,NaN,MA,Amherst,NaN,NaN,NaN
1,Entry-Level Cognitive / AI / Machine Learning ...,IBM,NaN,TX,Austin,NaN,NaN,NaN
2,PHP/MySQL Programmer,NaN,NaN,TX,Austin,NaN,NaN,NaN
3,Python Programmer Job,SAIC,NaN,CA,Monterey,NaN,NaN,NaN
4,BI Analyst,NaN,NaN,CA,San Francisco,NaN,NaN,NaN


In [3]:
merged_dataset.to_csv("final_dataframe.csv")

In [4]:
skill_list =['data analysis','machine learning','statistics','computer science','communication','mathematics', 'visualization', 'ai',\
             'deep learning', 'nlp', 'software development','neural networks', 'project management', 'software engineering', \
              'data engineering','bi','modeling','etl','statistical analysis','research','deep learning','interpersonal skills',\
            'web scraping','neural network','decision tree','random forest','testing']
tool_list = ['python','r','sql','hadoop','spark','java','sas','tableau','hive','scala','aws','c++','matlab',\
             'c','excel','nosql','linux','vba','json','numpy','pandas','matplotlib','api','mysql','mongodb','html','css',\
             'java script','ajax','d3','leaflet','matlab']
edu_list = ['bachelor','bachelors','master','masters','phd','post graduation','graduation','high school','bs','ms','ba','ma']

In [6]:
# Viv - the code can be improved to bring effeciency and can have a function to do the reg expression, howver for now use this and we can make improvements later
description = merged_dataset['Description']
skills = merged_dataset['Skills']
title = merged_dataset['Position']
desc_skills = description.append(skills)
parse_list = description.append(title)
skill_count = {}
tool_count = {}
edu_count = {}
no_edu_count = 0
for entry in parse_list:
    try:
        entry = re.sub('[./\n\r(),:-]', ' ', str(entry))
#        print(entry)
        entry = re.sub('[^A-Za-z0-9+ ]', '', str(entry)).lower()
        entry = re.sub('data analytics', 'data analysis', str(entry))
        entry = re.sub('artificial intelligence', 'ai', str(entry))
#        print(entry)
        for skill in skill_list:
            if (f' {skill} ' in f' {entry} '):
                if skill not in skill_count:
                    skill_count[skill] = 1
                else:
                    skill_count[skill] += 1
        for tool in tool_list:
            if f' {tool} ' in f' {entry} ':
                if tool not in tool_count:
                    tool_count[tool] = 1
                else:
                    tool_count[tool] += 1   
    except:
        print("Skipping an entry due to bad data")
        continue
description2 = merged_dataset['Description']
for item in description2:
    try:
        item = re.sub('[./\n\r(),:-]', ' ', str(item))
        item = re.sub('[^A-Za-z0-9+ ]', '', str(item)).lower()
        item = re.sub('data analytics', 'data analysis', str(item))
        item = re.sub('artificial intelligence', 'ai', str(item))
        i = 0
        for edu in edu_list:
            if f' {edu} ' in f' {item} ':
                if edu not in edu_count:
                    edu_count[edu] = 1
                else:
                    edu_count[edu] += 1 
            else:
                i +=1
        if i==12: 
            no_edu_count +=1
    except:
        print("Skipping an entry due to bad data")
        continue
print(skill_count)
print(tool_count)
print(edu_count)
education_count = {
    'High School':edu_count['high school'],
    'Bachelors':edu_count['bachelors']+edu_count['bachelor']+edu_count['graduation']+edu_count['bs']+edu_count['ba'],
    'Masters':edu_count['masters']+edu_count['master']+edu_count['post graduation']+edu_count['ms']+edu_count['ma'],
    'Phd':edu_count['phd'],
    'No education mentioned':no_edu_count
}
print(education_count)

{'communication': 4873, 'testing': 2253, 'statistics': 2317, 'mathematics': 1367, 'research': 5532, 'software development': 1339, 'project management': 790, 'modeling': 2226, 'etl': 1314, 'interpersonal skills': 688, 'data analysis': 2469, 'visualization': 1093, 'statistical analysis': 694, 'bi': 966, 'machine learning': 3586, 'computer science': 3207, 'nlp': 402, 'software engineering': 588, 'ai': 1066, 'data engineering': 334, 'deep learning': 1378, 'neural network': 48, 'neural networks': 301, 'web scraping': 14, 'decision tree': 29, 'random forest': 72}
{'sql': 4199, 'sas': 867, 'excel': 1352, 'html': 414, 'css': 372, 'python': 3987, 'linux': 982, 'tableau': 853, 'c': 1321, 'java': 2099, 'api': 451, 'java script': 25, 'json': 184, 'aws': 999, 'spark': 1312, 'mongodb': 286, 'r': 2147, 'hadoop': 1748, 'hive': 808, 'c++': 1166, 'mysql': 540, 'matlab': 1242, 'vba': 152, 'nosql': 629, 'scala': 661, 'ajax': 76, 'd3': 174, 'numpy': 171, 'pandas': 201, 'matplotlib': 50, 'leaflet': 5}
{'ms'